In [1]:
# Terminal: pip install xltodict : transform xml to dict-like obeject 
import xmltodict
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# read xml file
with open('DPR_Playgrounds_001.xml') as fd:
    doc = xmltodict.parse(fd.read())

In [3]:
# transform xml to dic, then to pd.DateFrame
doc_len = len(doc['playgrounds']['facility'])
b = pd.DataFrame()
for i in xrange(doc_len):
    a = doc['playgrounds']['facility'][i]
    b = b.append(pd.Series(dict(a)), ignore_index=True)

In [4]:
# transform 'lat' and 'lon' to numeric
b['lat'] = pd.to_numeric(b['lat'])
b['lon'] = pd.to_numeric(b['lon'])
#dropna
row1 = b.shape[0]
b = b.dropna(subset = ['lat', 'lon'])
row2 = b.shape[0]
print 'delete {} NA rows'.format(row1-row2)
# create 'geometry'
b["geometry"] = b.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)

delete 21 NA rows


In [5]:
# create GeoDataFrame
c = gpd.GeoDataFrame(b, geometry="geometry")
# set coordinate system
c.crs = {"init": "epsg:4326"}

In [6]:
c.head(2)

,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,geometry
0,N,None,None,"Noble, Franklin, Milton Sts",American Playground,B001,B001,None,None,40.7288,-73.9579,POINT (-73.9579 40.7288)
1,Y,N,4,BAY PKWY & CROPSEY AVENUE,Bensonhurst Park,B007-01,B007,None,None,40.5969,-73.9998,POINT (-73.99979999999999 40.5969)


In [7]:
# create shapefile and related files
c.to_file("playground.shp")